# Introduction to Modeling Libraries in Python

 This chapter introduces features of pandas that are helful when moving back and forth between wrangling and fitting, and also
 gives a short intro to statsmodels and scikit-learn.

 ### Interfacing Between pandas and Model Code

In [23]:
import pandas as pd
import numpy as np

In [24]:
# The point of contact between pandas and other analysis libraries is NumPy arrays.
data = pd.DataFrame({
    'x0': [1, 2, 3, 4, 5],
    'x1': [0.01, -0.01, 0.25, -4.1, 0.],
    'y': [-1.5, 0., 3.6, 1.3, -2.]
})

data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [25]:
data.columns

Index(['x0', 'x1', 'y'], dtype='object')

In [26]:
data.to_numpy()

array([[ 1.  ,  0.01, -1.5 ],
       [ 2.  , -0.01,  0.  ],
       [ 3.  ,  0.25,  3.6 ],
       [ 4.  , -4.1 ,  1.3 ],
       [ 5.  ,  0.  , -2.  ]])

In [27]:
type(data.to_numpy())

numpy.ndarray

In [28]:
# We can then convert back to a DataFrame by passing the NumPy array and column index
df2 = pd.DataFrame(data.to_numpy(), columns = ['one', 'two', 'three'])
df2

,one,two,three
0,1.0,0.01,-1.5
1,2.0,-0.01,0.0
2,3.0,0.25,3.6
3,4.0,-4.10,1.3
4,5.0,0.00,-2.0


In [29]:
# If the data is not homogeneous then using to_numpy will give an ndarray of Python objects
df3 = data.copy()
df3['strings'] = ['a', 'b', 'c', 'd', 'e']
df3

,x0,x1,y,strings
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,c
3,4,-4.10,1.3,d
4,5,0.00,-2.0,e


In [30]:
df3.to_numpy()

array([[1, 0.01, -1.5, 'a'],
       [2, -0.01, 0.0, 'b'],
       [3, 0.25, 3.6, 'c'],
       [4, -4.1, 1.3, 'd'],
       [5, 0.0, -2.0, 'e']], dtype=object)

In [31]:
type(df3.to_numpy())

numpy.ndarray

In [32]:
# loc indexing allows us to use only a subset of the columns
model_cols = ['x0', 'x1']
data.loc[:, model_cols].to_numpy()

array([[ 1.  ,  0.01],
       [ 2.  , -0.01],
       [ 3.  ,  0.25],
       [ 4.  , -4.1 ],
       [ 5.  ,  0.  ]])

In [33]:
# We can use the pandas.get_dummies method to address an array in which certain columns contain
# categorical data
data['category'] = pd.Categorical(['a', 'b', 'a', 'a', 'b'], categories=['a', 'b'])
data

,x0,x1,y,category
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,a
3,4,-4.10,1.3,a
4,5,0.00,-2.0,b


In [34]:
dummies = pd.get_dummies(data.category, prefix='category', dtype=float)
data_with_dummies = data.drop('category', axis=1).join(dummies)
data_with_dummies

,x0,x1,y,category_a,category_b
0,1,0.01,-1.5,1.0,0.0
1,2,-0.01,0.0,0.0,1.0
2,3,0.25,3.6,1.0,0.0
3,4,-4.10,1.3,1.0,0.0
4,5,0.00,-2.0,0.0,1.0


### Creating Model Descriptions with Patsy

Patsy describes statistical models with a string-based formula syntax. The syntax looks like
y ~ x0 + x1
where x0 and x1 are terms in the design matrix for the model.

In [35]:
data = pd.DataFrame({'x0': [1, 2, 3, 4, 5],
                     'x1': [0.01, -0.01, 0.25, -4.1, 0.],
                     'y': [-1.5, 0., 3.6, 1.3, -2.]})

data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [36]:
import patsy

y, X = patsy.dmatrices('y ~ x0 + x1', data)

y

DesignMatrix with shape (5, 1)
     y
  -1.5
   0.0
   3.6
   1.3
  -2.0
  Terms:
    'y' (column 0)

In [37]:
X

DesignMatrix with shape (5, 3)
  Intercept  x0     x1
          1   1   0.01
          1   2  -0.01
          1   3   0.25
          1   4  -4.10
          1   5   0.00
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'x1' (column 2)

In [38]:
# A Patsy DesignMatrix instance is just a NumPy ndarray with added metadata
np.asarray(X)

array([[ 1.  ,  1.  ,  0.01],
       [ 1.  ,  2.  , -0.01],
       [ 1.  ,  3.  ,  0.25],
       [ 1.  ,  4.  , -4.1 ],
       [ 1.  ,  5.  ,  0.  ]])

In [39]:
# Adding +0 to the model will suppress the Intercept term
patsy.dmatrices('y ~ x0 + x1 + 0', data)[1]

DesignMatrix with shape (5, 2)
  x0     x1
   1   0.01
   2  -0.01
   3   0.25
   4  -4.10
   5   0.00
  Terms:
    'x0' (column 0)
    'x1' (column 1)

In [40]:
# The Patsy objects can be passed directly into numpy algorithms like linalg.lstsq (ordinary least squares)
coef, resid, _, _ = np.linalg.lstsq(X, y, rcond=None)

In [42]:
# The metadata is retained here in the design_info attribute
coef

array([[ 0.31290976],
       [-0.07910564],
       [-0.26546384]])

In [43]:
coef = pd.Series(coef.squeeze(), index = X.design_info.column_names)
coef

Intercept    0.312910
x0          -0.079106
x1          -0.265464
dtype: float64

Data Transformations in Patsy Formulas

In [44]:
# You can put Python code into Patsy formulas
y, X = patsy.dmatrices('y ~ x0 + np.log(np.abs(x1) + 1)', data)

X

DesignMatrix with shape (5, 3)
  Intercept  x0  np.log(np.abs(x1) + 1)
          1   1                 0.00995
          1   2                 0.00995
          1   3                 0.22314
          1   4                 1.62924
          1   5                 0.00000
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'np.log(np.abs(x1) + 1)' (column 2)

In [47]:
# Patsy has built-in support for standardizing (to mean 0 and variance 1) and centering (subtracting the mean)
y, X = patsy.dmatrices('y ~ standardize(x0) + center(x1)', data)

X

DesignMatrix with shape (5, 3)
  Intercept  standardize(x0)  center(x1)
          1         -1.41421        0.78
          1         -0.70711        0.76
          1          0.00000        1.02
          1          0.70711       -3.33
          1          1.41421        0.77
  Terms:
    'Intercept' (column 0)
    'standardize(x0)' (column 1)
    'center(x1)' (column 2)

In [50]:
# If you tranform a training dataset, you'll need to also transform the test dataset. Patsy supports
# saving the transformations applied in-sample to apply later to out-of-sample data.
new_data = pd.DataFrame({
    'x0': [6, 7, 8, 9],
    'x1': [3.1, -0.5, 0, 2.3],
    'y': [1, 2, 3, 4]
})

new_X = patsy.build_design_matrices([X.design_info], new_data)
new_X

[DesignMatrix with shape (4, 3)
   Intercept  standardize(x0)  center(x1)
           1          2.12132        3.87
           1          2.82843        0.27
           1          3.53553        0.77
           1          4.24264        3.07
   Terms:
     'Intercept' (column 0)
     'standardize(x0)' (column 1)
     'center(x1)' (column 2)]

In [51]:
# Adding columns from a dataset by name requires wrapping them in the special I function
y, X = patsy.dmatrices('y ~ I(x0 + x1)', data)

X

DesignMatrix with shape (5, 2)
  Intercept  I(x0 + x1)
          1        1.01
          1        1.99
          1        3.25
          1       -0.10
          1        5.00
  Terms:
    'Intercept' (column 0)
    'I(x0 + x1)' (column 1)

Categorical Data and Patsy

In [53]:
# Nonnumeric data can be transformed in different ways. In Patsy nonnumeric terms are converted to 
# dummy variables by default.
data = pd.DataFrame({
    'key1': ['a', 'a', 'b', 'b', 'a', 'b', 'a', 'b'],
    'key2': [0, 1, 0, 1, 0, 1, 0, 0],
    'v1': [1, 2, 3, 4, 5, 6, 7, 8],
    'v2': [-1, 0, 2.5, -0.5, 4.0, -1.2, 0.2, -1.7]
})

y, X = patsy.dmatrices('v2 ~ key1', data)

X

DesignMatrix with shape (8, 2)
  Intercept  key1[T.b]
          1          0
          1          0
          1          1
          1          1
          1          0
          1          1
          1          0
          1          1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)

In [55]:
# Omitting the intercept causes the category values to be included in the model design matrix
y, X = patsy.dmatrices('v2 ~ key1 + 0', data)

X

DesignMatrix with shape (8, 2)
  key1[a]  key1[b]
        1        0
        1        0
        0        1
        0        1
        1        0
        0        1
        1        0
        0        1
  Terms:
    'key1' (columns 0:2)

In [56]:
# You can also transform numerical data to categorical with the C function
y, X = patsy.dmatrices('v2 ~ C(key2)', data)

X

DesignMatrix with shape (8, 2)
  Intercept  C(key2)[T.1]
          1             0
          1             1
          1             0
          1             1
          1             0
          1             1
          1             0
          1             0
  Terms:
    'Intercept' (column 0)
    'C(key2)' (column 1)

### Introduction to statsmodels

The statsmodels library contains tools for implementing many frequentist statistical methods.

In [64]:
# Linear models in statsmodels can be interfaced either through arrays or formulas, which are imported
# through API modules.

import statsmodels.api as sm
import statsmodels.formula.api as smf

In [65]:
# We'll generate a linear model from random data
rng = np.random.default_rng(seed=12345)

def dnorm(mean, variance, size=1):
    if isinstance(size, int):
        size = size
    return mean + np.sqrt(variance)*rng.standard_normal(size)

N = 100
X = np.c_[dnorm(0, 0.4, size=N),
          dnorm(0, 0.6, size=N),
          dnorm(0, 0.2, size=N)]
eps = dnorm(0, 0.1, size=N)
beta = [0.1, 0.3, 0.5]

y = np.dot(X, beta) + eps

In [66]:
X

array([[-9.00506021e-01, -1.89429577e-01, -1.02787020e+00],
       [ 7.99252054e-01, -1.54598388e+00, -3.27397080e-01],
       [-5.50654833e-01, -1.20254287e-01,  3.29358994e-01],
       [-1.63915546e-01,  8.24039852e-01,  2.08274848e-01],
       [-4.76512913e-02, -2.13146980e-01, -4.82436357e-02],
       [-4.68576597e-01, -1.43558784e+00, -1.52694953e-01],
       [-8.65068061e-01, -9.63148432e-02,  7.08625055e-01],
       [ 4.10395842e-01,  6.08038650e-01,  1.26222105e-01],
       [ 2.28353201e-01,  1.56467440e-01,  4.06761512e-01],
       [-1.23509905e+00, -3.31585038e-01,  1.76681376e-01],
       [ 1.48463222e+00,  1.43167842e+00, -2.99354280e-01],
       [ 6.12531226e-01,  1.47169718e+00,  6.95582154e-01],
       [-4.80278623e-01, -7.62397041e-02, -5.53712608e-01],
       [ 5.70600290e-01,  6.30092128e-01, -5.34945034e-01],
       [-2.95327118e-01,  3.04024846e-01, -1.91921494e-01],
       [-3.83834219e-02,  6.05303068e-01, -3.26313824e-01],
       [ 4.98908970e-01,  1.12569928e+00

In [67]:
y

array([-0.59952668, -0.58845445,  0.18563386, -0.00747657, -0.01537445,
       -0.48405182,  0.03006418,  0.21745535,  0.09733398,  0.29428061,
       -0.55821726,  0.39275937, -0.88717278, -0.14095687, -0.24884462,
       -0.11545172,  0.49031861, -0.53928623,  0.01003077, -0.12181392,
       -0.40652191, -0.26296953,  0.24121365, -0.01486882, -0.8269326 ,
        0.85796862, -0.1581605 ,  0.3229089 , -0.3182448 , -0.25177682,
        0.01201277, -0.27692688,  0.48915537,  0.02713607,  0.32623478,
       -0.67005187, -0.43638036,  0.19876113,  0.29108143,  1.22925781,
       -0.13454699,  0.11618133, -0.28334439,  0.82639714,  0.65173309,
        0.36932726,  0.46060306, -0.36001815, -0.67943794, -0.32391053,
        0.22890353,  0.33392927, -0.02893472,  0.3514912 ,  0.4104761 ,
        0.02342595, -0.08816253, -0.42223253,  0.95031167, -0.84319881,
       -0.17742481, -0.58276729, -0.04786858,  0.49981461, -0.40998529,
       -0.06505591, -0.11920059, -0.73782536,  0.11294051, -0.50

In [68]:
# The function sm.add_constant adds and intercept column to the data matrix
X_model = sm.add_constant(X)

X_model[:5]

array([[ 1.        , -0.90050602, -0.18942958, -1.0278702 ],
       [ 1.        ,  0.79925205, -1.54598388, -0.32739708],
       [ 1.        , -0.55065483, -0.12025429,  0.32935899],
       [ 1.        , -0.16391555,  0.82403985,  0.20827485],
       [ 1.        , -0.04765129, -0.21314698, -0.04824364]])

In [70]:
# Now the sm.OLS class can be used to fit an ordinary least squares linear regression
model = sm.OLS(y, X)

# The fit method returns a regression results object
results = model.fit()
results.params

array([0.06681503, 0.26803235, 0.45052319])

In [72]:
# The summary method prints detailed diagnostics of the model
print(results.summary())

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.469
Model:                            OLS   Adj. R-squared (uncentered):              0.452
Method:                 Least Squares   F-statistic:                              28.51
Date:                Fri, 21 Mar 2025   Prob (F-statistic):                    2.66e-13
Time:                        11:41:45   Log-Likelihood:                         -25.611
No. Observations:                 100   AIC:                                      57.22
Df Residuals:                      97   BIC:                                      65.04
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [73]:
# If the model parameters and data are instead in a DataFrame, we can pass using the formula api
data = pd.DataFrame(X, columns=['col0', 'col1', 'col2'])
data['y'] = y
data[:5]

,col0,col1,col2,y
0,-0.900506,-0.189430,-1.027870,-0.599527
1,0.799252,-1.545984,-0.327397,-0.588454
2,-0.550655,-0.120254,0.329359,0.185634
3,-0.163916,0.824040,0.208275,-0.007477
4,-0.047651,-0.213147,-0.048244,-0.015374


In [74]:
results = smf.ols('y ~ col0 + col1 + col2', data=data).fit()
results.params

Intercept   -0.020799
col0         0.065813
col1         0.268970
col2         0.449419
dtype: float64

In [75]:
# We can also predict values using the trained model
results.predict(data[:5])

0   -0.592959
1   -0.531160
2    0.058636
3    0.283658
4   -0.102947
dtype: float64

Estimating Time Series Processes

In [77]:
# Simulate time series data with autoregressive structure and noise

init_x = 4

values = [init_x, init_x]
N = 1000

b0 = 0.8
b1 = -0.4
noise = dnorm(0, 0.1, N)
for i in range(N):
    new_x = values[-1]*b0 + values[-2]*b1 + noise[i]
    values.append(new_x)

In [79]:
# You might not know the number of lags in an AR model, so you can fit it with a large value
from statsmodels.tsa.ar_model import AutoReg

MAXLAGS = 5
model = AutoReg(values, MAXLAGS)
results = model.fit()

/home/zieglers/Documents/python/pandas_mckinney/.venv/lib64/python3.6/site-packages/statsmodels/tsa/ar_model.py:252: FutureWarning: The parameter names will change after 0.12 is released. Set old_names to False to use the new names now. Set old_names to True to use the old names. 
  FutureWarning,


In [81]:
# The results show the intercept first followed by the estimates of the lags
# Note the second and third entries give good estimates of b0 and b1
results.params

array([ 0.02346612,  0.8096828 , -0.42865278, -0.03336517,  0.04267874,
       -0.05671529])

### Introduction to scikit-learn

This section is a brief intro to the scikit-learn API and the use of pandas with scikit-learn.

In [84]:
# We'll use a dataset from a Kaggle competition about passenger survival rates on the Titanic
train = pd.read_csv("datasets/titanic/train.csv")
test = pd.read_csv("datasets/titanic/test.csv")

train.head(4)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S


In [85]:
# First we check to see if there is any missing data
train.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [86]:
test.isna().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [87]:
# Suppose we wanted to use age as a predictor of survival on the Titanic. We first need to fill the missing 
# data in the "age" column. Here's will use the median to fill the nulls in each table.
input_value = train['Age'].median()
train['Age'] = train['Age'].fillna(input_value)
test['Age'] = test['Age'].fillna(input_value)

In [88]:
# We'll add a column called "IsFemale" in order to encode Sex
train['IsFemale'] = (train['Sex'] == 'female').astype(int)
test['IsFemale'] = (test['Sex'] == 'female').astype(int)

In [89]:
# Next we'll create model variables and NumPy arrays
predictors = ['Pclass', 'IsFemale', 'Age']

X_train = train[predictors].to_numpy()
X_test = test[predictors].to_numpy()
y_train = train['Survived'].to_numpy()
X_train[:5]

array([[ 3.,  0., 22.],
       [ 1.,  1., 38.],
       [ 3.,  1., 26.],
       [ 1.,  1., 35.],
       [ 3.,  0., 35.]])

In [90]:
y_train[:5]

array([0, 1, 1, 1, 0])

In [95]:
import sklearn

from sklearn.linear_model import LogisticRegression
model = LogisticRegression()

In [96]:
# Now we'll fit the model to the training data
model.fit(X_train, y_train)
LogisticRegression()

LogisticRegression()

In [97]:
# Now we can use the trained model to make predictions
y_predict = model.predict(X_test)
y_predict[:10]

array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0])

In [98]:
# We can also apply cross-validation for parameter tuning to avoid overfitting.
# The LogisticRegressionCV class implements logistic regression and includes a parameter
# to indicate how fine-grained of a grid search to do.

from sklearn.linear_model import LogisticRegressionCV

model_cv = LogisticRegressionCV(Cs=10)
model_cv.fit(X_train, y_train)

LogisticRegressionCV()

In [99]:
# The helper function cross_val_score handles the data-splitting process for cross-validation
# For example we can cross-validate our model with four nonoverlapping splits of the training data:
from sklearn.model_selection import cross_val_score

model = LogisticRegression(C=10)

scores = cross_val_score(model, X_train, y_train, cv=4)
scores

array([0.77578475, 0.79820628, 0.77578475, 0.78828829])

Additional books to consider:

    Introduction to Machine Learning with Python by Andreas Müller and Sarah Guido (O'Reilly)

    Python Data Science Handbook by Jake VanderPlas (O'Reilly)

    Data Science from Scratch: First Principles with Python by Joel Grus (O'Reilly)

    Python Machine Learning by Sebastian Raschka and Vahid Mirjalili (Packt Publishing)

    Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow by Aurélien Géron (O'Reilly)
